In [94]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_mclaren_owner', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4048 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_mclaren_owner...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [93]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [45]:
post_start_date = "2022-09-07"
post_gcoin_end_date = "2022-11-02"
post_craft_end_date = "2022-11-07"

itemdesc.12021002 - standard black  
itemdesc.12021003 - standard white  
itemdesc.12021001 - elite yellow

In [46]:
craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", post_start_date, post_craft_end_date)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41485)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41485)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [47]:
special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", post_start_date, post_craft_end_date).where(col("Currency").like("%token%"))

In [48]:
standard_black = craft.where(col("ItemDescId") == "itemvoucherdesc.12021002")
standard_white = special_craft.where(col("ItemId") == "itemdesc.12021003")
elite_yellow = special_craft.where(col("ItemId") == "itemdesc.12021001")

In [24]:
standard_black.select(count("*"), countDistinct("AccountId")).show(truncate=False)

+--------+-------------------------+
|count(1)|count(DISTINCT AccountId)|
+--------+-------------------------+
|90106   |84655                    |
+--------+-------------------------+



In [30]:
standard_black.groupBy("AccountId").agg(count("*").alias("cnt")).groupBy("cnt").agg(countDistinct("AccountId").alias("user_cnt")).orderBy("cnt").show(truncate=False)

+---+--------+
|cnt|user_cnt|
+---+--------+
|1  |79830   |
|2  |4319    |
|3  |412     |
|4  |74      |
|5  |15      |
|6  |4       |
|7  |1       |
+---+--------+



In [25]:
standard_white.select(count("*"), countDistinct("AccountId")).show(truncate=False)

+--------+-------------------------+
|count(1)|count(DISTINCT AccountId)|
+--------+-------------------------+
|268579  |268578                   |
+--------+-------------------------+



In [20]:
elite_yellow.select(count("*"), countDistinct("AccountId")).show(truncate=False)

+--------+-------------------------+
|count(1)|count(DISTINCT AccountId)|
+--------+-------------------------+
|62049   |62049                    |
+--------+-------------------------+



In [49]:
mclaren_user = standard_black.groupBy("AccountId").agg(min("date").alias("own_date")).withColumn("mclaren", lit("standard_black")) \
    .unionByName(standard_white.groupBy("AccountId").agg(min("date").alias("own_date")).withColumn("mclaren", lit("standard_white"))) \
    .unionByName(elite_yellow.groupBy("AccountId").agg(min("date").alias("own_date")).withColumn("mclaren", lit("elite_yellow")))

In [29]:
mclaren_user.select(countDistinct("AccountId"), count("*")).show(truncate=False)

+-------------------------+--------+
|count(DISTINCT AccountId)|count(1)|
+-------------------------+--------+
|380462                   |415282  |
+-------------------------+--------+



In [52]:
mysql.insert_table(mclaren_user, "labs", "mclaren_owner")

In [95]:
mclaren_user = mysql.read_table(spark, "labs", "mclaren_owner")

In [96]:
mclaren_distinct_user = mclaren_user.groupBy("AccountId").agg(min("own_date").alias("first_own_date"))

In [98]:
user_login = load_data_mart('pc', post_start_date, post_craft_end_date, "user_master").where(col("lastlogindate") == col("date")).withColumnRenamed("accountid", "account_id")

In [100]:
from pyspark.sql.window import Window
Window_Spec  = Window.partitionBy("account_id").orderBy("date")
login_with_num = user_login.withColumn("login_date_num", row_number().over(Window_Spec))

In [101]:
mclaren_distinct_user = mclaren_distinct_user.withColumnRenamed("AccountId", "account_id").withColumnRenamed("first_own_date", "date")
login_with_num = login_with_num.select("date", "account_id", "login_date_num")

In [102]:
mclaren_own_login_date_num = mclaren_distinct_user.join(login_with_num, \
                           ["account_id", "date"], "left")

In [73]:
mclaren_own_login_date_num.select(count("*"), countDistinct("account_id")).show(truncate=False)

+--------+--------------------------+
|count(1)|count(DISTINCT account_id)|
+--------+--------------------------+
|380462  |380462                    |
+--------+--------------------------+



In [33]:
mclaren_user.groupBy("AccountId").agg(count("*").alias("mclaren_cnt")).groupBy("mclaren_cnt").agg(countDistinct("AccountId").alias("user_cnt")).orderBy("mclaren_cnt").show(truncate=False)

+-----------+--------+
|mclaren_cnt|user_cnt|
+-----------+--------+
|1          |346515  |
|2          |33074   |
|3          |873     |
+-----------+--------+



In [103]:
tm_user = mclaren_user.groupBy("AccountId").agg(count("*").alias("mclaren_cnt")).filter(col("mclaren_cnt") > 1)

In [35]:
tm_user.select(countDistinct("AccountId"), count("*")).show(truncate=False)

+-------------------------+--------+
|count(DISTINCT AccountId)|count(1)|
+-------------------------+--------+
|33947                    |33947   |
+-------------------------+--------+



In [106]:
one_mclaren_user = mclaren_user.join(tm_user.select("AccountId"), "AccountId", "leftanti")

In [107]:
sb_user = one_mclaren_user.where(col("mclaren") == "standard_black")
sw_user = one_mclaren_user.where(col("mclaren") == "standard_white")
ey_user = one_mclaren_user.where(col("mclaren") == "elite_yellow")

In [108]:
sb_user.select(countDistinct("AccountId"), count("*")).show(truncate=False)

+-------------------------+--------+
|count(DISTINCT AccountId)|count(1)|
+-------------------------+--------+
|51067                    |51067   |
+-------------------------+--------+



In [109]:
sw_user.select(countDistinct("AccountId"), count("*")).show(truncate=False)

+-------------------------+--------+
|count(DISTINCT AccountId)|count(1)|
+-------------------------+--------+
|259736                   |259736  |
+-------------------------+--------+



In [110]:
ey_user.select(countDistinct("AccountId"), count("*")).show(truncate=False)

+-------------------------+--------+
|count(DISTINCT AccountId)|count(1)|
+-------------------------+--------+
|35712                    |35712   |
+-------------------------+--------+



In [111]:
mclaren_user_group = sb_user.select("AccountId").withColumn("mclaren_group", lit("SB")) \
    .unionByName(sw_user.select("AccountId").withColumn("mclaren_group", lit("SW"))) \
    .unionByName(ey_user.select("AccountId").withColumn("mclaren_group", lit("EY"))) \
    .unionByName(tm_user.select("AccountId").withColumn("mclaren_group", lit("TM")))

In [114]:
mclaren_user_group.count()

380462

In [115]:
mclaren_own_login_date_num = mclaren_own_login_date_num.withColumnRenamed("login_date_num", "mclaren_own_date_num")

In [80]:
mclaren_own_login_date_num.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- mclaren_own_date_num: integer (nullable = true)



In [116]:
mclaren_user_group = mclaren_user_group.join(mclaren_own_login_date_num.withColumnRenamed("account_id", "AccountId").select("AccountId", "mclaren_own_date_num"), \
                                            "AccountId")

In [118]:
mysql.insert_table(mclaren_user_group, "labs", "mclaren_user_group")